##### Imports

In [263]:
import pandas as pd
from pathlib import Path

In [264]:
root = Path.cwd().parent
data_path = root / "data/aps_2425/stata/stata13/apsp_o24s25_eul_pwta22.dta"

##### Initial exploration

In [265]:
df = pd.read_stata(data_path, convert_categoricals=False)

In [266]:
df
# Training in past 3 months on current underemployment hours
# https://datacatalogue.ukdataservice.ac.uk/studies/study/9500#details
# https://doc.ukdataservice.ac.uk/doc/9500/mrdoc/pdf/9500_lfs_user_guide_vol3_variabledetails2022js.pdf
# https://doc.ukdataservice.ac.uk/doc/9500/mrdoc/pdf/9500_lfs_user_guide_vol6_aps_user_guide_2024.pdf
# https://doc.ukdataservice.ac.uk/doc/9500/mrdoc/pdf/9500_lfs_user_guide_vol2_questionnaire_2022jscombined.pdf
# https://doc.ukdataservice.ac.uk/doc/9500/mrdoc/pdf/9500_lfs_user_guide_vol6_aps_user_guide_2024.pdf

,AAGE,ACTHR,ACTHR2,ACTPOT,ACTUOT,ACTWKDY1,ACTWKDY2,ACTWKDY3,ACTWKDY4,ACTWKDY5,...,XDISDDA,Y2JOB,YLESS20,YMORE,YPAYL20,YPAYM,YPTJOB,YSTART,YTETJB,YVARY99
0,12,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
1,12,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
2,4,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
3,13,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
4,8,30.0,-9.0,0.0,0.0,2,3,5,-9,-9,...,-9,-9,13,-9,-9,-9,-9,-9,-9,-9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176025,9,-9.0,-9.0,-9.0,-9.0,1,2,3,4,5,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
176026,2,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
176027,13,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9
176028,13,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,-9,-9,-9,-9,-9,-9,-9,-9,-9,-9


In [267]:
df = df.drop(columns='idref')

In [268]:
df.rename(columns={'UNDHRS': 'Underemployment hours',
                   'ED13WK': 'Training (last 3 months)'}, inplace=True)

In [269]:
df['Underemployment hours'].value_counts().sort_index()

Underemployment hours
-9.0     169490
 1.0         37
 1.5          3
 2.0        175
 2.5         33
          ...  
 56.0         1
 60.0         1
 72.0         1
 97.0         7
 99.0       274
Name: count, Length: 88, dtype: int64

In [270]:
df['Training (last 3 months)'].value_counts().sort_index()

Training (last 3 months)
-9    99794
-8      194
 1    19670
 2    56372
Name: count, dtype: int64

In [271]:
df.dtypes.value_counts()

int8       493
int16       19
float64     16
str         11
int32        8
Name: count, dtype: int64

In [272]:
cols_to_adjust = list(df.select_dtypes(exclude=['number']).columns)

In [273]:
df.loc[:, cols_to_adjust].head()

,BANDG,BANDG2,BANDN,BANDN2,CTRY9D,FDSNGDEG,GOR9DCENSUS2021,PARK,REFDTE,SNGDEGN,SNGHD
0,-9,-9,-9,-9,E92000001,-9,E12000009,E65000001,06102024,-9,-9
1,-9,-9,-9,-9,E92000001,-9,E12000009,E65000001,06102024,-9,-9
2,-9,-9,-9,-9,E92000001,-9,E12000009,E65000001,06102024,-9,-9
3,-9,-9,-9,-9,E92000001,-9,E12000009,E65000001,06102024,-9,-9
4,-9,-9,-9,-9,E92000001,-9,E12000009,E65000001,06102024,-9,-9


In [274]:
cols_to_float = ['BANDG', 'BANDG2', 'BANDN', 'BANDN2']
df[cols_to_float] = df[cols_to_float].astype('float64')

In [275]:
cols_to_drop = [col for col in cols_to_adjust if col not in cols_to_float]
df = df.drop(columns=cols_to_drop, errors='ignore')

In [276]:
df = df.drop(columns=df.columns[((df == -8) | (df == -9)).sum() / len(df) > 0.5].difference(['Underemployment hours', 'Training (last 3 months)']))
# Drop cols with more than 50% missing data

In [277]:
df = df[~((df == -8) | (df == -9)).any(axis=1)]
age_cols = df.columns.difference(['age', 'aage'])
df = df[~(df[age_cols] == 99).any(axis=1)]

---

In [278]:
set(df.nunique().sort_values(ascending=False))

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 31, 39, 48, 65, 830, 862, 1173}

In [279]:
df.nunique()[df.nunique().between(31, 1173)].sort_values(ascending=False)
#

NPWT22                   1173
PWTA22                    862
PIWTA22                   830
Underemployment hours      65
AGE                        48
HIQUAL22                   39
HITQUA15                   31
REFWKD                     31
dtype: int64

In [280]:
df = df.drop(columns=['NPWT22', 'PWTA22', 'PIWTA22'])

In [281]:
df.nunique()[df.nunique().between(2, 15)].sort_values(ascending=False)
#

LKWFWM            15
CRYOX7_EUL_Sub    14
NSECM20           14
REFWKM            12
edageband         12
                  ..
QUAL21_11          2
QUAL21_13          2
QUAL21_14          2
QUAL21_15          2
WRKING             2
Length: 96, dtype: int64

In [282]:
cols_to_bin = list(df.columns[df.nunique().between(2, 15)].difference(['Training (last 3 months)']))
for col in cols_to_bin:
    df = pd.concat([df, pd.get_dummies(pd.cut(df[col], bins=min(5, df[col].nunique())), prefix=f'{col}_bin', dtype=int)], axis=1)
    df = df.drop(columns=col)

In [283]:
df['Training (last 3 months)'] = df['Training (last 3 months)'].replace(2, 0)

In [286]:
vars_of_interest = ['Underemployment hours', 'Training (last 3 months)']
other_vars = sorted([col for col in df.columns if col not in vars_of_interest])
df = df[vars_of_interest + other_vars]

In [287]:
df.head()

,Underemployment hours,Training (last 3 months),"AAGE_bin_(1.99, 4.0]","AAGE_bin_(10.0, 12.0]","AAGE_bin_(4.0, 6.0]","AAGE_bin_(6.0, 8.0]","AAGE_bin_(8.0, 10.0]",AGE,"ANXIOUS_bin_(-0.01, 2.0]","ANXIOUS_bin_(2.0, 4.0]",...,"WORTH_bin_(4.0, 6.0]","WORTH_bin_(6.0, 8.0]","WORTH_bin_(8.0, 10.0]","WRKING_bin_(0.999, 1.5]","WRKING_bin_(1.5, 2.0]","edageband_bin_(10.916, 27.8]","edageband_bin_(27.8, 44.6]","edageband_bin_(44.6, 61.4]","edageband_bin_(61.4, 78.2]","edageband_bin_(78.2, 95.0]"
20,10.0,1,0,0,0,1,0,44,0,0,...,0,1,0,1,0,1,0,0,0,0
137,5.0,0,0,0,0,1,0,39,1,0,...,0,1,0,1,0,1,0,0,0,0
156,3.0,0,0,0,0,0,1,53,1,0,...,0,0,1,1,0,1,0,0,0,0
231,5.0,0,0,0,1,0,0,29,0,0,...,0,1,0,1,0,1,0,0,0,0
269,8.0,0,0,0,1,0,0,28,1,0,...,0,1,0,1,0,1,0,0,0,0


---

In [114]:
df.loc[:, df.columns[df.max().between(10, 1900, inclusive='neither')]].head()

,AAGE,ACTHR,ACTHR2,ACTPOT,ACTUOT,AGE,BACTHR,BUSHR,CAMEMT,CONMON,...,TOTUS2,TTACHR,TTUSHR,TYEMPS,TYPSCH12,Underemployment hours,UOTHR,USUHR,YLESS20,YPAYL20
0,12,-9.0,-9.0,-9.0,-9.0,64,-9,-9,-9,-9,...,-9.0,-9,-9,-9,-9,-9.0,-9.0,-9.0,-9,-9
1,12,-9.0,-9.0,-9.0,-9.0,63,-9,-9,-9,-9,...,-9.0,-9,-9,-9,-9,-9.0,-9.0,-9.0,-9,-9
2,4,-9.0,-9.0,-9.0,-9.0,24,-9,-9,-9,-9,...,-9.0,-9,-9,-9,-9,-9.0,-9.0,-9.0,-9,-9
3,13,-9.0,-9.0,-9.0,-9.0,73,-9,-9,-9,-9,...,-9.0,-9,-9,-9,-9,-9.0,-9.0,-9.0,-9,-9
4,8,30.0,-9.0,0.0,0.0,44,30,40,-9,-9,...,45.0,30,45,-9,-9,-9.0,0.0,40.0,13,-9


In [112]:
max_vals = df.max()

NTNLTY12    997.0
CRY12       997.0
SC20LMN     926.0
SC20MMN     926.0
SC20SMN     926.0
            ...  
LKTIMA       11.0
LKTIMB       11.0
NVQAF15      11.0
REDYL13      11.0
ERNCM06      11.0
Length: 112, dtype: float64

In [117]:
max_vals[max_vals>=90]

AGE                          99.0
CAMEYR                     2025.0
CAMEYR2                    2025.0
CONMPY                     2025.0
CONSEY                     2025.0
CRY12                       997.0
edageband                    95.0
EMPMON                      912.0
GROSS99                   41000.0
GRSEXP                    41000.0
GRSPRD                       97.0
GRSSWK                      788.0
GRSSWK2                     788.0
idref                    176030.0
LEFTYR                     2025.0
NET99                     41000.0
NETPRD                       97.0
NETWK                       788.0
NETWK2                      788.0
NPWT22                    32698.0
NTNLTY12                    997.0
OVHRS                        97.0
PIWTA22                    8870.0
PWTA22                     7213.0
REFWKY                     2025.0
SC20LMN                     926.0
SC20MMN                     926.0
SC20SMN                     926.0
SCHM12                       97.0
SCNTGA        

In [92]:
numeric_cols

,AAGE,ACTHR,ACTHR2,ACTPOT,ACTUOT,ACTWKDY1,ACTWKDY2,ACTWKDY3,ACTWKDY4,ACTWKDY5,...,SNGHD_9.4.1,SNGHD_9.4.2,SNGHD_9.4.2.1,SNGHD_9.4.2.2,SNGHD_9.4.3,SNGHD_9.4.5,SNGHD_9.4.6,SNGHD_9.4.9,SNGHD_9.9,SNGHD_9.9.9
0,12,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
1,12,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
2,4,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
3,13,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
4,8,30.0,-9.0,0.0,0.0,2,3,5,-9,-9,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176025,9,-9.0,-9.0,-9.0,-9.0,1,2,3,4,5,...,0,0,0,0,0,0,0,0,0,0
176026,2,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
176027,13,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
176028,13,-9.0,-9.0,-9.0,-9.0,-9,-9,-9,-9,-9,...,0,0,0,0,0,0,0,0,0,0
